
opengl context?

PKG_CONFIG = SDL2


In [ ]:

#include <iostream>
//#include <glad/glad.h>
#include <SDL.h>


#if __cplusplus
extern "C" {
#endif

static SDL_Window* window;

int createWindow(void) {
	window = nullptr;
	
	int screen_width = 1000;
	int screen_height = 800;

	if (SDL_Init(SDL_INIT_VIDEO) < 0) {
		std::cout << "SDL failed initialization. " << SDL_GetError();
		return 0;
	} else {
		std::cout << "SDL successfully initialized.";
	}

	SDL_GL_SetAttribute(SDL_GL_CONTEXT_MAJOR_VERSION, 4);
	SDL_GL_SetAttribute(SDL_GL_CONTEXT_MINOR_VERSION, 1);
	SDL_GL_SetAttribute(SDL_GL_CONTEXT_PROFILE_MASK, SDL_GL_CONTEXT_PROFILE_CORE);
	SDL_GL_SetAttribute(SDL_GL_DOUBLEBUFFER, 1);
	SDL_GL_SetAttribute(SDL_GL_DEPTH_SIZE, 24);

	window = SDL_CreateWindow("C++ SDL2 Window | OpenGL", 
			0, 
			0, 
			screen_width, 
			screen_height, 
			SDL_WINDOW_SHOWN | SDL_WINDOW_OPENGL);
	SDL_GLContext context;
	context = SDL_GL_CreateContext(window);

	bool done = false;
	
	while(!done) {
		SDL_Event event;
		while(SDL_PollEvent(&event)){
			if(event.type == SDL_QUIT) {
				done = true;
			}
		}
//		glClear(GL_DEPTH_BUFFER_BIT | GL_COLOR_BUFFER_BIT);
		//SDL_GL_SwapWindow(window);
		done = true;
		break;
	}
	//SDL_DestroyWindow(window);
	//SDL_Quit();

  return SDL_GetWindowID(window);
}

#if __cplusplus
}
#endif



test opengl renderer?


In [ ]:


let beginFrame, endFrame
async function doFrame(window) {
  if(!beginFrame) {
    frameLoader = await importer.import('opengl frame')
    beginFrame = frameLoader.beginFrame
    endFrame = frameLoader.endFrame
  }
  beginFrame(window)

  endFrame(window)
}

async function testOpenGL() {
  const {createWindow} = await importer.import('opengl context')
  //const { createShaderProgram, createQuad } = await importer.import('shader toy')
  let window = createWindow()
  //createShaderProgram();
  //createQuad();

  //console.log(window)
  doFrame(window)
  setInterval(doFrame.bind(null, window), 10)
}

module.exports = testOpenGL



test shader toy?


In [ ]:


let renderLoop, endFrame
async function doFrame(window) {
  if(!renderLoop) {
    frameLoader = await importer.import('shader toy')
    renderLoop = frameLoader.renderLoop
    endFrame = frameLoader.endFrame
  }
  renderLoop(window)

  endFrame(window)
}

async function testOpenGL() {
  const {createWindow} = await importer.import('opengl context')
  const { createShaderProgram, createQuad } = await importer.import('shader toy')
  const cellCode = importer.interpret('fragment shader')
  let window = createWindow()
  createShaderProgram(
`
#version 330 core
out vec2 uv;
void main() {
    vec2 positions[3] = vec2[3](
        vec2(-1.0, -1.0),
        vec2( 3.0, -1.0),
        vec2(-1.0,  3.0)
    );
    gl_Position = vec4(positions[gl_VertexID], 0.0, 1.0);
}
`,
`
#version 330 core
out vec4 FragColor;
void main() {
    FragColor = vec4(1.0, 0.0, 0.0, 1.0); // Pure Red
}
`
  //  cellCode.code
  );
  //createQuad();

  //console.log(window)
  doFrame(window)
  setInterval(doFrame.bind(null, window), 10)
}

module.exports = testOpenGL



opengl frame?

PKG_CONFIG = SDL2


In [ ]:
#include <vector>
#include <cmath>
#include <SDL.h>

class Point {
  public:
    double x, y, z;

    Point() {
        this->x = 0;
        this->y = 0;
        this->z = 0;
    }

    Point(double x, double y, double z) {
        this->x = x;
        this->y = y;
        this->z = z;
    }

    Point(double x, double y) {
        this->x = x;
        this->y = y;
        this->z = 0;
    }

    double operator[](int i) const {
        if (i == 0) return this->x;
        if (i == 1) return this->y;
        return this->z;
    }

    double& operator[](int i) {
        if (i == 0) return this->x;
        if (i == 1) return this->y;
        return this->z;
    }
};

typedef std::vector<double> Vector;
typedef std::vector<Vector> Matrix;

Matrix dot(const Matrix& a, const Matrix& b) {
    Matrix result = Matrix(a.size(), Vector(b[0].size(), 0));
    for (int i=0; i<a.size(); i++) {
        for (int j=0; j<b[0].size(); j++) {
            for (int k=0; k<b.size(); k++) {
                result[i][j] += a[i][k] * b[k][j];
            }
        }
    }
    return result;
}

Point transform(const Matrix& matrix, const Point& point) {
    Matrix p = {{point.x}, {point.y}, {point.z}};
    Matrix r = dot(matrix, p);
    return Point(r[0][0], r[1][0], r[2][0]);
}

Point translate(const Point& shift, const Point& point) {
    return Point(
        point.x + shift.x,
        point.y + shift.y,
        point.z + shift.z
    );
}

void connect(SDL_Renderer* const renderer, const std::vector<Point> &points, int i, int j) {
    SDL_RenderDrawLine(
        renderer,
        points[i].x,
        points[i].y,
        points[j].x,
        points[j].y
    );
}

Matrix getRotationMatrix() {
    double alpha = 0.001;
    Matrix rotationX = {
        {1, 0, 0},
        {0, cos(alpha), -sin(alpha)},
        {0, sin(alpha), cos(alpha)}
    };

    double beta = 0.002;
    Matrix rotationY = {
        {cos(beta), 0, sin(beta)},
        {0, 1, 0},
        {-sin(beta), 0, cos(beta)}
    };

    double gamma = 0.003;
    Matrix rotationZ = {
        {cos(gamma), -sin(gamma), 0},
        {sin(gamma), cos(gamma), 0},
        {0, 0, 1}
    };

    return dot(rotationZ, dot(rotationY, rotationX));
}
  

#if __cplusplus
extern "C" {
#endif


int WIDTH = 1000;
int HEIGHT = 800;

static bool first = false;
static SDL_Renderer* renderer;
std::vector<Point> points;
Matrix rotationXYZ;

Point screenShift;
Point screenShiftOpposite;

void beginFrame(int windowId) {
    SDL_Window *window = SDL_GetWindowFromID(windowId);
    SDL_Event event;
    while(SDL_PollEvent(&event)){
        if(event.type == SDL_QUIT) {
            SDL_DestroyWindow(window);
            SDL_Quit();
            return;
        }
    }
  if(!first) {
    first = true;
    
    renderer = SDL_CreateRenderer(
        window,
        -1,
        SDL_RENDERER_ACCELERATED
    );

    points = {
        Point(-1, 1, 1),
        Point(1, 1, 1),
        Point(1, -1, 1),
        Point(-1, -1, 1),
        Point(-1, 1, -1),
        Point(1, 1, -1),
        Point(1, -1, -1),
        Point(-1, -1, -1)
    };

    screenShift = Point(WIDTH / 2, HEIGHT / 2);
    screenShiftOpposite = Point(-WIDTH / 2, -HEIGHT / 2);
    int scale = 200;

    for (Point& p : points) {
        p.x = (scale * p.x + screenShift.x);
        p.y = (scale * p.y + screenShift.y);
        p.z = (scale * p.z + screenShift.z);
    }

    rotationXYZ = getRotationMatrix();
  }


  for (Point &p : points) {
    p = translate(screenShiftOpposite, p);
    p = transform(rotationXYZ, p);
    p = translate(screenShift, p);
  }

  SDL_SetRenderDrawColor(renderer, 255, 255, 255, 255);
  SDL_RenderClear(renderer);
  SDL_SetRenderDrawColor(renderer, 0, 0, 0, 255);
  for (int i=0; i<4; i++) {
      connect(renderer, points, i, (i + 1) % 4);
      connect(renderer, points, i + 4, ((i + 1) % 4) + 4);
      connect(renderer, points, i, i + 4);
  }
  SDL_RenderPresent(renderer);
  SDL_Delay(3);
}

void endFrame(int windowId) {
    SDL_Window *window = SDL_GetWindowFromID(windowId);
  SDL_GL_SwapWindow(window);
}


#if __cplusplus
}
#endif




shader toy?

PKG_CONFIG = SDL2

FRAMEWORKS = OpenGL


In [ ]:
#include <SDL.h>
#if defined(__APPLE__)
#include <OpenGL/gl.h>
#include <OpenGL/glext.h>
#else
#include <GL/glew.h>
#endif

typedef struct {
  float x, y, z;
} vec3;

typedef struct {
  float x, y, z, w;
} vec4;


#define SCREEN_WIDTH 1000
#define SCREEN_HEIGHT 800

//SDL_Window* window = NULL;
SDL_GLContext glContext;
GLuint shaderProgram;
GLuint quadVAO;

const char* vertexShaderSource = "#version 330 core\n"
    "in vec2 pos;\n"
    "out vec2 uv;\n"
    "void main() {\n"
    "    vec2 pos = vec2((gl_VertexID & 1) * 2.0 - 1.0, (gl_VertexID >> 1) * 2.0 - 1.0);\n"
    "    uv = pos * 0.5 + 0.5;\n"
    "    gl_Position = vec4(pos.xy,0.0,1.0);\n"
    "}";

const char* fragmentShaderSource = "#version 330 core\n"
    "out vec4 FragColor;\n"
    "in vec2 texCoords;\n"
    "uniform float iTime;\n"
    "void main() {\n"
    "    vec2 uv = texCoords * 2.0 - 1.0;\n"
    "    float color = 0.5 + 0.5 * sin(iTime + uv.x * 10.0);\n"
    "    FragColor = vec4(color, color, color, 1.0);\n"
    "}";

#if __cplusplus
extern "C" {
#endif
    
GLuint compileShader(GLenum type, const char* source) {
  GLuint shader = glCreateShader(type);
  glShaderSource(shader, 1, &source, NULL);
  glCompileShader(shader);

  // Check for errors
  GLint success;
  glGetShaderiv(shader, GL_COMPILE_STATUS, &success);
  if (!success) {
      char infoLog[512];
      glGetShaderInfoLog(shader, 512, NULL, infoLog);
      printf("Shader compilation error: %s\n", infoLog);
  }
  return shader;
}

float startTime;

void createShaderProgram(char *vertexSource, char *shaderSource) {
  GLuint vertexShader = compileShader(GL_VERTEX_SHADER, vertexSource);
  GLuint fragmentShader = compileShader(GL_FRAGMENT_SHADER, shaderSource);

  shaderProgram = glCreateProgram();
  glAttachShader(shaderProgram, vertexShader);
  glAttachShader(shaderProgram, fragmentShader);
  glLinkProgram(shaderProgram);

  // Check for linking errors
  GLint success;
  glGetProgramiv(shaderProgram, GL_LINK_STATUS, &success);
  if (!success) {
      char infoLog[512];
      glGetProgramInfoLog(shaderProgram, 512, NULL, infoLog);
      printf("Shader linking error: %s\n", infoLog);
  }

  glDeleteShader(vertexShader);
  glDeleteShader(fragmentShader);
  startTime = (float) SDL_GetTicks() / 1000.0f;
}

void createQuad() {
  float quadVertices[] = {
      // x, y
      -1.0f, -1.0f,
       1.0f, -1.0f,
      -1.0f,  1.0f,
      -1.0f,  1.0f,
       1.0f, -1.0f,
       1.0f,  1.0f
  };

  GLuint VBO;
  glGenVertexArraysAPPLE(1, &quadVAO);
  glGenBuffers(1, &VBO);

  glBindVertexArrayAPPLE(quadVAO);
  glBindBuffer(GL_ARRAY_BUFFER, VBO);
  glBufferData(GL_ARRAY_BUFFER, sizeof(quadVertices), quadVertices, GL_STATIC_DRAW);
  glVertexAttribPointer(0, 2, GL_FLOAT, GL_FALSE, 2 * sizeof(float), (void*)0);
  glEnableVertexAttribArray(0);

  glBindBuffer(GL_ARRAY_BUFFER, 0);
  glBindVertexArrayAPPLE(0);
}

int frame = 0;
float prevTime = 0.0f;
float iChannelTime[4] = {0.0f, 0.0f, 0.0f, 0.0f};
vec4 iMouse = {0.0f, 0.0f, 0.0f, 0.0f};  // Store mouse position
vec3 iChannelResolution[4] = {
  {0.0f, 0.0f, 0.0f},
  {0.0f, 0.0f, 0.0f},
  {0.0f, 0.0f, 0.0f},
  {0.0f, 0.0f, 0.0f},
};

void renderLoop(int windowId) {
  //int running = 1;
  SDL_Event event;
  SDL_Window *window = SDL_GetWindowFromID(windowId);

  glViewport(0, 0, SCREEN_WIDTH, SCREEN_HEIGHT);


  //while (running) {
    while (SDL_PollEvent(&event)) {
        if (event.type == SDL_QUIT) {
          SDL_DestroyWindow(window);
          SDL_Quit();
          return;
        }
    }

    float currentTime = (float) SDL_GetTicks() / 1000.0f - startTime;
    float iTimeDelta = currentTime - prevTime;
    prevTime = currentTime;
    float iFrameRate = (iTimeDelta > 0.0f) ? (1.0f / iTimeDelta) : 0.0f;
    frame++;
  
    time_t t = time(NULL);
    struct tm *tm_info = localtime(&t);
    float iDate[4] = { (float)tm_info->tm_year + 1900,  // Year
                        (float)tm_info->tm_mon + 1,      // Month
                        (float)tm_info->tm_mday,         // Day
                        (float)tm_info->tm_hour * 3600.0f + tm_info->tm_min * 60.0f + tm_info->tm_sec }; // Time in seconds

    // Render
    glClear(GL_COLOR_BUFFER_BIT);

    glUseProgram(shaderProgram);
    //GLint currentProgram;
    //glGetIntegerv(GL_CURRENT_PROGRAM, &currentProgram);
    //printf("Current Shader Program: %d\n", currentProgram);

    glUniform3f(glGetUniformLocation(shaderProgram, "iResolution"), SCREEN_WIDTH, SCREEN_HEIGHT, 1.0f);
    glUniform1f(glGetUniformLocation(shaderProgram, "iTime"), currentTime);
    glUniform1f(glGetUniformLocation(shaderProgram, "iTimeDelta"), iTimeDelta);
    glUniform1f(glGetUniformLocation(shaderProgram, "iFrameRate"), iFrameRate);
    glUniform1i(glGetUniformLocation(shaderProgram, "iFrame"), frame);
    glUniform1fv(glGetUniformLocation(shaderProgram, "iChannelTime"), 4, iChannelTime);
    glUniform3fv(glGetUniformLocation(shaderProgram, "iChannelResolution"), 4, (float*)iChannelResolution);
    glUniform4f(glGetUniformLocation(shaderProgram, "iMouse"), iMouse.x, iMouse.y, iMouse.z, iMouse.w);
    glUniform4fv(glGetUniformLocation(shaderProgram, "iDate"), 1, iDate);
    glUniform1f(glGetUniformLocation(shaderProgram, "iSampleRate"), 44100.0f);

    glBindVertexArrayAPPLE(0);
    //glBindVertexArrayAPPLE(quadVAO);
    //glDrawArrays(GL_TRIANGLES, 0, 6);
    /*
    glBegin(GL_TRIANGLES);
    glVertex2f(-1.0f, -1.0f);
    glVertex2f( 3.0f, -1.0f);
    glVertex2f(-1.0f,  3.0f);
    glEnd();
*/
    glDrawArrays(GL_TRIANGLES, 0, 3);

      //SDL_GL_SwapWindow(window);
  //}
}

void endFrame(int windowId) {
  SDL_Window *window = SDL_GetWindowFromID(windowId);
  SDL_GL_SwapWindow(window);
}

/*

void cleanup() {
  glDeleteVertexArrays(1, &quadVAO);
  glDeleteProgram(shaderProgram);
  SDL_GL_DeleteContext(glContext);
  SDL_DestroyWindow(window);
  SDL_Quit();
}

int main(int argc, char* argv[]) {
  initSDL();
  createShaderProgram();
  createQuad();
  renderLoop();
  cleanup();
  return 0;
}
*/

#if __cplusplus
}
#endif



fragment shader?

https://www.shadertoy.com/view/lsfGRr



In [ ]:
#version 330 core
uniform vec3      iResolution;           // viewport resolution (in pixels)
uniform float     iTime;                 // shader playback time (in seconds)
uniform float     iTimeDelta;            // render time (in seconds)
uniform float     iFrameRate;            // shader frame rate
uniform int       iFrame;                // shader playback frame
uniform float     iChannelTime[4];       // channel playback time (in seconds)
uniform vec3      iChannelResolution[4]; // channel resolution (in pixels)
uniform vec4      iMouse;                // mouse pixel coords. xy: current (if MLB down), zw: click
// uniform samplerXX iChannel0..3;          // input channel. XX = 2D/Cube
uniform vec4      iDate;                 // (year, month, day, time in seconds)
uniform float     iSampleRate;           // sound sample rate (i.e., 44100)
out vec4 fragColor;
in vec2 fragCoord;

const mat2 m = mat2( 0.80,  0.60, -0.60,  0.80 );

float hash( float n )
{
    return fract(sin(n)*43758.5453);
}

float noise( in vec2 x )
{
    vec2 i = floor(x);
    vec2 f = fract(x);

    f = f*f*(3.0-2.0*f);

    float n = i.x + i.y*57.0;

    return mix(mix( hash(n+ 0.0), hash(n+ 1.0),f.x),
               mix( hash(n+57.0), hash(n+58.0),f.x),f.y);
}

float fbm( vec2 p )
{
    float f = 0.0;
    f += 0.50000*noise( p ); p = m*p*2.02;
    f += 0.25000*noise( p ); p = m*p*2.03;
    f += 0.12500*noise( p ); p = m*p*2.01;
    f += 0.06250*noise( p ); p = m*p*2.04;
    f += 0.03125*noise( p );
    return f/0.984375;
}

float length2( vec2 p )
{
    vec2 q = p*p*p*p;
    return pow( q.x + q.y, 1.0/4.0 );
}

void main( )
{
    vec2 p = (2.0*fragCoord-iResolution.xy)/iResolution.y;

    // polar coordinates
    float r = length( p );
    float a = atan( p.y, p.x );

    // animate
    r *= 1.0 + 0.2*clamp(1.0-r,0.0,1.0)*sin(4.0*iTime);

    // iris (blue-green)
    vec3 col = vec3( 0.0, 0.3, 0.4 );
    float f = fbm( 5.0*p );
    col = mix( col, vec3(0.2,0.5,0.4), f );
    
    // yellow towards center
    col = mix( col, vec3(0.9,0.6,0.2), 1.0-smoothstep(0.2,0.6,r) );

    // darkening
    f = smoothstep( 0.4, 0.9, fbm( vec2(15.0*a,10.0*r) ) );
    col *= 1.0-0.5*f;

    // distort
    a += 0.05*fbm( 20.0*p );

    // cornea
    f = smoothstep( 0.3, 1.0, fbm( vec2(20.0*a,6.0*r) ) );
    col = mix( col, vec3(1.0,1.0,1.0), f );

    // edges
    col *= 1.0-0.25*smoothstep( 0.6,0.8,r );

    // highlight
    f = 1.0-smoothstep( 0.0, 0.6, length2( mat2(0.6,0.8,-0.8,0.6)*(p-vec2(0.3,0.5) )*vec2(1.0,2.0)) );
    col += vec3(1.0,0.9,0.9)*f*0.985;
    
    // shadow
    col *= vec3(0.8+0.2*cos(r*a));

    // pupil
    f = 1.0-smoothstep( 0.2, 0.25, r );
    col = mix( col, vec3(0.0), f );

    // crop
    f = smoothstep( 0.79, 0.82, r );
    col = mix( col, vec3(1.0), f );

    // vignetting
    vec2 q = fragCoord/iResolution.xy;
    col *= 0.5 + 0.5*pow(16.0*q.x*q.y*(1.0-q.x)*(1.0-q.y),0.1);
 
	fragColor = vec4( col, 1.0 );
}
